Note that the labelling of these sections is based on the version of the paper completed on 15/08/22. This cell should be updated in the event of a change to the labelling in the paper.

Currently running on Sagemath version 9.4.rc0, Release Date: 2021-07-27.
Using Python 3.9.5.

This notebook is intended to be ran sequentially from the beginning. Changing the order in which cells are ran may lead to errors. 

## Initial Calculations

In [47]:
# We start by just verifying the resultant stated. 
K.<z> = CyclotomicField(5)
P.<X,Y,Z> = ProjectiveSpace(K, 2)
R.<x,y> = QQ[]

F = X*(Y^5 + Z^5) + (X*Y*Z)^2 - X^4*Y*Z - 2*(Y*Z)^3
f = F(x, y, 1).change_ring(QQ)
res = f.resultant(f.derivative(y), y)
print(res.factor())

(-1) * (x - 1)^2 * x^4 * (x^4 + x^3 + x^2 + x + 1)^2 * (256*x^10 - 837*x^5 + 3456)


## Lemma 2.4

In [48]:
# We can then verify that the singular points on the curve, at least over the field QQ[zeta],
# are those claimed. 
C = Curve(F)
claimed_singular_points = set([P([z^k, z^(2*k), 1]) for k in range(5)] + [P([1, 0, 0])])
print(set(C.singular_points())==claimed_singular_points)

True


## Corollay 2.6

In [49]:
C = Curve(f)
F = C.function_field()
divx = F(x).divisor()
# Note we use our knowledge of the valuation at c,d,b,a to assign them. 
# We will then check that this agrees with the divisor of y. 
supp = divx.support()
c, d, b, a = sorted(supp, key = lambda si: divx.valuation(si))
print(F(y).divisor()==1*a-1*b-3*c+3*d)

True


## Lemma 2.8

In [50]:
# We check that the condition for the quadric Q holds as claimed. 
v1, v2, v3, v4 = [y^3 - x, y^2*x - 1, y - x^2, y*(x^2 - y)]
R_quotient = R.quotient(R.ideal(f))
print(R_quotient(v1*v2 + v3*v4)==0)

True


## Proposition 2.10

In [51]:
# We take the canonical divisor to be the divisor associated to the differential v3.
# We then check a stronger condition than Prop 2.10, namely that the divisor of this differential is
# *exactly* the divisor a+2b+3c.
K_can = F(v3).divisor() + F(x).differential().divisor() - F(f.derivative(y)).divisor()
print(K_can - (1*a + 2*b + 3*c)==0)

True


## Proposition 2.11

In [59]:
# We first verify the cubic that the La satisfy. 
# In order to use one of the standard orderings on polynomial ring generators in Sage, 
# we take X=x0, Y=x1, Z=x2. 
R.<y1, y2, y3, y4, x0, x1, x2> = PolynomialRing(QQ, order='invlex')
L1 = x0^2*x1 - x1^2*x2
L2 = x0^2*x2 - x1*x2^2
L3 = x0*x1^2 - x2^3
L4 = x0*x2^2 - x1^3
II = ideal(y1-L1, y2-L2, y3-L3, y4-L4)
B = II.groebner_basis()

# Because of the ordering, any relation between the La will come out as a relation between the ya
# This follows the logic of "Inversion of Birational Maps with Groebner Bases", Schicho, 1997.
supset = set([y1, y2, y3, y4])
for bi in B:
    if supset.issuperset(set(bi.variables())):
        print(bi)

y2*y4^2 - y1^2*y4 - y1*y3^2 + y2^2*y3


In [53]:
# We now use Macaulay2, interfaced through Sage, to invert the rational map. 
# See the Cremona package for an explanation of output. 
macaulay2.eval("""
               loadPackage "Cremona";
               Qx = QQ[x0, x1, x2];
               Qy = QQ[y1, y2, y3, y4];
               phi = map(Qx, Qy, matrix{{x0^2*x1-x1^2*x2,x0^2*x2-x1*x2^2,x0*x1^2-x2^3,x0*x2^2-x1^3}});
               Quot = Qy/image(rationalMap(phi));
               psi = map(Qx, Quot, matrix{{x0^2*x1-x1^2*x2,x0^2*x2-x1*x2^2,x0*x1^2-x2^3,x0*x2^2-x1^3}});
               isBirational(psi, MathMode=>true)
               psiInv = inverseMap(psi)
               """
              )

/usr/share/Macaulay2/Cremona.m2:10:1:(3):[8]: error: package Cremona not reloaded; try Reload => true



o1000000029 : RingMap Qx <--- Qy


o1000000031 : RingMap Qx <--- Quot
MathMode: output certified!

o1000000032 = true

                                2
o1000000033 = map (Quot, Qx, {y2  - y1*y3, y1*y4, y2*y4})

o1000000033 : RingMap Quot <--- Qx

In [58]:
# We now want to construct the map from the cubic we found, namely 
# L2*L4^2 - L1^2*L4 - L1*L3^2 + L2^2*L3, to the Clebsch surface. 
# To do so we need the Eckardt points in the L coordinates
# We will do this as follows:
#  - construct the lines in the HC model.
#  - find the intersection of lines that don't share a vertex in the HC model, using Groebnber bases. 
#  - map these to the La coordinates

# We construct the lines in P = K[X,Y,Z] that connect two of the singular points 
# Note here we will now treat P as a polynomial ring not a projective space. 
P.<X,Y,Z> = K[]
# These lines connect [1:0:0] and [1:z^k:z^(-k)]
lines = [z^(-k)*Y-z^k*Z for k in range(5)]
# These lines connect [1:z^k:z^(-k)] and [1:z^l:z^(-l)]
lines += [(z^(l-k)-z^(k-l))*X - (z^(-k)-z^(-l))*Y + (z^k-z^l)*Z
          for k in range(5) for l in range(k+1,5)]
# We record the pairs of indices corresponding to these lines, using the numbering of Lemma 2.4
indices = [set((k, 5)) for k in range(5)] + [set((k, l)) for k in range(5) for l in range(k+1, 5)]

# We will loop through the lines we have, so we record the number of these. 
nl = len(lines)
if nl!=15:
    raise ValueError("Incorrect number of lines.")

# The La as written below will effect the transformation in the loop. 
L1 = X^2*Y - Y^2*Z
L2 = X^2*Z - Y*Z^2
L3 = X*Y^2 - Z^3
L4 = X*Z^2 - Y^3
P3 = ProjectiveSpace(3, K, names='y')

# E_points will store the Eckardt points. 
E_points = []

# We will want to find the intersecion in affine coordinates. 
# As all the singular points are defined in the chart where X!=0, 
# we take affine coordinates u=Y/Z and v=Z/X.
# Note we avoid z as to not overwrite the notation of the cyclotomic element. 
R.<u, v> = K[]
# We loop over ordered triples of distinct lines
for i in range(nl):
    for j in range(i+1, nl):
        for k in range(j+1, nl):
            # If the lines share a vertex, then the intersection will not be what we call 
            # and Eckardt point, so continue. 
            if (indices[i].intersection(indices[j])).intersection(indices[k])!=set():
                continue
            
            # If the lines don't intersect in a line, then continue as there is no
            # associated Eckardt point.
            II = P.ideal([lines[i], lines[j], lines[k]])
            if II.dimension() != 1:
                continue
            # Construct the intersection as the variety of the (now zero dimensional) ideal.
            Var = R.ideal([bi.subs({X:1, Y:u, Z:v})
                   for bi in II.groebner_basis()]).variety()
            # Read out the (first) point in the variety and append the corresponding L points. 
            yi = Var[0][u]
            zi = Var[0][v]
            E_points.append(P3([L1(1, yi, zi), L2(1, yi, zi), L3(1, yi, zi), L4(1, yi, zi)]))

if len(E_points)!=10:
    raise ValueError("Insufficiently many Eckardt points found.")

# We will also take E_points_canonical to be the known Eckardt points of the curve when written in 
# standard coordinates. 
E_points_canonical = [[1,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,0,1]]
E_points_canonical += [[1,-1,0,0], [1,0,-1,0], [1,0,0,-1], [0,1,-1,0], [0,1,0,-1], [0,0,1,-1]]
E_points_canonical = [P3(Ei) for Ei in E_points_canonical]

# We now want to find a projective transformation that sends E_points to E_points_canonical

# We will do this by mapping a maximal linearly independent amount of E_points to the 
# same number of E_points_canonical, then check that the remaining points are correctly
# sent over. 

# What we are doing here is emulating the method inside point_transormation_matrix, 
# but making it more efficient by finding one set of source points that work, and then 
# not rechecking that this is a goot set and finding it's matrix each time. 
# Note the matrix produce by this method should act by multiplying from the left. 
import itertools
n = 3

def get_matrix(S, N):
    a = matrix(N+1, N+1, [S[j][i] for i in range(N+1) for j in range(N+1)])
    b = matrix(N+1, 1, list(S[N+1]))
    X = a.solve_right(b)
    m = matrix(N+1, N+1, [X[i,0]*S[i][j] for i in range(N+1) for j in range(N+1)])
    return m.transpose()

for comb in itertools.combinations(E_points, n+2):
    # Check that the source points are linearly independent
    if any(l == 0 for l in matrix(K, [list(s) for s in comb]).minors(n+1)):
        continue
    # msi will map E_points in comb to the standard points related to get_matrix. 
    msi = get_matrix(comb, n).inverse()
    # sources will be the remaining E_points we need to check are correctly mapped. 
    sources = [Ei for Ei in E_points if Ei not in comb]
    break

print("Valid transform matrix:")
for comb in itertools.combinations(E_points_canonical, n+2):
    # Check that the target points are linearly independent
    if any(l == 0 for l in matrix(K, [list(t) for t in comb]).minors(n+1)):
        continue
    # target_set will be the remaining E_points_canonical. 
    target_set = set([Ei for Ei in E_points_canonical if Ei not in comb])
    
    # Looping over permutations of the chosen E_points_canonical, 
    # map these to the standard points, and check that the combined transform
    # has the desired effect on sources. 
    for pi in Permutations(comb):
        m_target = get_matrix(pi, n)
        
        return_mat = m_target*msi
        image_set = set([return_mat*ri for ri in sources])

        if image_set != target_set:
            continue

        X = return_mat
        last_row = list(X.rows()[-1])[:]
        last_ele = last_row.pop()
        while last_ele == 0:
            last_ele = last_row.pop()
        X *= ZZ(1)/last_ele
        print(X)
        # For illustrative purposes, we will only print one possible matrix
        # then break out of the two loops. One could remove these statements 
        # to see all possible transform matrices, then choose a desired one
        # as we did. 
        break
    break

Valid transform matrix:
[                 z                 -1  z^3 + z^2 + z + 1                z^2]
[-z^3 - z^2 - z - 1               -z^2                 -1                  z]
[               z^3               -z^3               -z^3                z^3]
[               z^2  z^3 + z^2 + z + 1                 -z                  1]


In [62]:
# We now take a particular A that is valid, and check that imposing F=0 gives the quadric. 
# This is the same calculation as before, now we just go first to a quotient ring where
# F=0. 
A = matrix([[ z^3,   -1, -z^2,    z], [   1, -z^3,   -z,  z^2], 
            [ z^2,   -z,   -1,  z^3], [   z, -z^2, -z^3,    1]])
Ainv = A.inverse()

F = x0*(x1^5+x2^5)+(x0*x1*x2)^2-x0^4*x1*x2-2*(x1*x2)^3
R_Quotient.<y1, y2, y3, y4, X, Y, Z> = R.quotient(F)
L1 = X^2*Y - Y^2*Z
L2 = X^2*Z - Y*Z^2
L3 = X*Y^2 - Z^3
L4 = X*Z^2 - Y^3
vx = A*vector([L1, L2, L3, L4])
II = ideal(y1-vx[0], y2-vx[1], y3-vx[2], y4-vx[3])
B = II.groebner_basis()
supset = set([y1, y2, y3, y4])
for bi in B:
    if supset.issuperset(set(bi.variables())):
        print(bi)

y2^3 + y2^2*y3 + y2*y3^2 + y3^3 + y2^2*y4 + y2*y3*y4 + y3^2*y4 + y2*y4^2 + y3*y4^2 + y4^3
y1^2 + y1*y2 + y2^2 + y1*y3 + y2*y3 + y3^2 + y1*y4 + y2*y4 + y3*y4 + y4^2
